In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
from selenium import webdriver

# URL 리스트
urls = {
    '2022': "http://auto.danawa.com/newcar/?Work=record&Tab=Grand&Month=2022-01-00&MonthTo=2022-12-00",
    '2023': "http://auto.danawa.com/newcar/?Work=record&Tab=Grand&Month=2023-01-00&MonthTo=2023-12-00",
    '2024': "http://auto.danawa.com/newcar/?Work=record&Tab=Grand&Month=2024-01-00&MonthTo=2024-12-00"
}

# 헤더 설정
headers = {
    'User-Agent': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'
}

# 연도별 데이터를 담을 딕셔너리
data_by_year = {}

wd = webdriver.Chrome()

# 각 연도별로 데이터 가져오기
for year, url in urls.items():
    wd.get(url)
    html = wd.page_source
    if True:
        soup = BeautifulSoup(html, 'html.parser')
        # 제목(차종) 리스트 
        list_title = []
        for i in range(1, 7):
            tt_lists = soup.select(f'#autodanawa_gridC > div.gridMain > article > main > div:nth-child(1) > table > tbody > tr:nth-child({i}) > td.title')
            title = tt_lists[0].text.strip()
            list_title.append(title)
        # 판매량 리스트
        list_sales = []
        for i in range(1, 7):
            tr_lists = soup.select(f'#autodanawa_gridC > div.gridMain > article > main > div:nth-child(1) > table > tbody > tr:nth-child({i}) > td.num')
            sales = int(tr_lists[0].text.strip().replace(",", ""))
            list_sales.append(sales)
        
        # 제목과 판매량을 합친 딕셔너리 생성
        dict_ts = dict(zip(list_title, list_sales))
        
        # 연도별 데이터를 딕셔너리에 추가
        data_by_year[year] = dict_ts 

# 연도별 데이터를 데이터프레임으로 변환
df = pd.DataFrame.from_dict(data_by_year, orient='index')

# 기아, 현대, 제네시스, 쉐보레 열만 선택
df_filtered = df[['현대', '기아', '제네시스', '쉐보레']]

# 결과 출력
df_filtered.to_csv('car_sales_data.csv', encoding='utf-8-sig')
df_filtered

,현대,기아,제네시스,쉐보레
2022,536532,540181,135044,37236
2023,620482,564523,126567,38306
2024,359635,361703,88655,17050
